### Scraping NodeFlair Job Postings using BeautifulSoup

In [9]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install pandas

# !pip install selenium
# !pip install webdriver-manager

In [158]:
import time
from datetime import datetime, timedelta
import requests 
import pandas as pd

from bs4 import BeautifulSoup 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

#### Input Search URL and get HTML
- Use selenium to load the page because the webpage loads the data dynamically (i.e. the full HTML will not be returned by using BeautifulSoup)

In [159]:
# chromedriver_path = "C:/Users/user/chromedriver.exe"
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [119]:
num_pages = 247

# Initialise list to store details of each individual job listing post
metadata = []

# Iterate through all pages
for page_num in range(1, num_pages+1):
    url = f"https://www.nodeflair.com/jobs?query=cloud&page={page_num}&sort_by=recent#"
    driver.get(url) 
    time.sleep(3)

    # Get Soup using BeautifulSoup after loading complete HTML
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    # Find all job posting cards
    job_listings = soup.find_all("div", class_="col-12 col-sm-12 col-md-6 col-lg-4")
    
    # Iterate through all (12) job listings on a page to get link to individual listing and date posted
    for listing in job_listings:
        job_url = listing.find("a").get("href")
        date = listing.find("p", {"style": "font-style: italic; color: rgb(131, 131, 131);"}).get_text()

        metadata.append([job_url, date])

In [121]:
# Convert each job posting to a row in a DataFrame
job_listings_df = pd.DataFrame(data = metadata, columns=["URL", "Date posted"])
job_listings_df

,URL,Date posted
0,/jobs/53907,about 11 hours ago
1,/jobs/53898,about 11 hours ago
2,/jobs/53894,about 11 hours ago
3,/jobs/53893,about 11 hours ago
4,/jobs/53891,about 11 hours ago
...,...,...
2959,/jobs/46008,about 2 months ago
2960,/jobs/46001,about 2 months ago
2961,/jobs/45989,about 2 months ago
2962,/jobs/45982,about 2 months ago


In [122]:
job_listings_df.to_csv("nodeflair_links.csv")

#### Function to get data from each job posting
- Job Title ✔
- Company ✔
- Salary ??
- Job Type ??
- Seniority ✔
- Years of Experience ??
- Tech Stack ✔
- Job Description ✔
  - Click Read More 

In [123]:
def getJobTitle(soup):
    job_title = soup.find("h1", class_="job-title-name").get_text()
    return job_title

def getCompanyName(soup):
    company_name = soup.find("h2", class_="company-title-name").get_text()
    return company_name

def getSeniorityTechStacks(soup):
    # Both Seniority and Tech Stack have the same HTML class 
    tags = [tag.get_text() for tag in soup.find_all("div", class_="grey-tag")]
    cleaned_tags = [tag.replace("\n", "").replace(" ", "") for tag in tags]

    seniority_tags = ["Mid", "Senior", "Lead", "Manager", "Director", "Principal", "Intern", "Junior"]
    seniority = []
    tech_stack = []

    # Differentiating between tags for Seniority and Tech Stack
    for tag in cleaned_tags:
        if tag in seniority_tags:
            seniority.append(tag)
        else:
            tech_stack.append(tag)

    return seniority, tech_stack

def getJobDesc(soup):
    job_description_container = soup.find("div", id="job-description")

    text_list = []
    for text in job_description_container.find_all("div"):
        text = text.get_text()
        if text != "":
            text_list.append(text)

    for text in job_description_container.find_all("ul"):
        points = text.find_all("li")
        for point in points:
            if point != "":
                text_list.append(point.get_text())

    return " ".join(text_list)

In [82]:
def getData(soup):
    job_title = getJobTitle(soup)
    company_name = getCompanyName(soup)
    seniority, techstack = getSeniorityTechStacks(soup)

    job_desc = getJobDesc(soup)

    data = [job_title, company_name, seniority, techstack, job_desc]

    return data

In [145]:
def scrapePost(job_url):
    print("scraping:", job_url)

    try: 
        new_url = f"https://www.nodeflair.com/{job_url}"
        driver.get(new_url) 

        # Manually expand the size of the container to display all text
        expand_text_toggle = driver.execute_script("document.getElementById('job-description').style.height = '1000px';")

        # Manually display all hidden text 
        read_more_toggle = driver.execute_script("document.getElementById('job-description').getAttribute('aria-expanded').innerHTML = 'true';")
        time.sleep(3)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        data = getData(soup)
    
    except Exception as e:
        print(e)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        data = getData(soup)
        
    print(" done scraping")
    return data

In [155]:
def getDatePosted(text):
    date_of_scraping = datetime.now()
    date_posted = ''

    if "hours" in text:
        date = date_of_scraping + timedelta(days=-1)
        date_posted = date.date().strftime('%y-%m-%d')
        
    elif "months" in text:
        n_months = text.split(" ")[1]
        date = date_of_scraping + timedelta(months=-n_months)
        date_posted = date.date().strftime('%y-%m-%d')

    return date_posted

In [162]:
job_listings_df = pd.read_csv("nodeflair_links.csv", index_col=0)
job_listings_df.head()

,URL,Date posted
0,/jobs/53907,about 11 hours ago
1,/jobs/53898,about 11 hours ago
2,/jobs/53894,about 11 hours ago
3,/jobs/53893,about 11 hours ago
4,/jobs/53891,about 11 hours ago


In [163]:
job_listings_df["data"] = job_listings_df["URL"].apply(scrapePost)

scraping: /jobs/53907
 done scraping
scraping: /jobs/53898
 done scraping
scraping: /jobs/53894
 done scraping
scraping: /jobs/53893
 done scraping
scraping: /jobs/53891
 done scraping
scraping: /jobs/53885
 done scraping
scraping: /jobs/53879
 done scraping
scraping: /jobs/53876
 done scraping
scraping: /jobs/53870
 done scraping
scraping: /jobs/53869
 done scraping
scraping: /jobs/53862
 done scraping
scraping: /jobs/53861
 done scraping
scraping: /jobs/53860
 done scraping
scraping: /jobs/53856
 done scraping
scraping: /jobs/53852
 done scraping
scraping: /jobs/53849
 done scraping
scraping: /jobs/53847
 done scraping
scraping: /jobs/53845
 done scraping
scraping: /jobs/53842
 done scraping
scraping: /jobs/53839
 done scraping
scraping: /jobs/53838
 done scraping
scraping: /jobs/53837
 done scraping
scraping: /jobs/53836
 done scraping
scraping: /jobs/53835
 done scraping
scraping: /jobs/53833
 done scraping
scraping: /jobs/53832
 done scraping
scraping: /jobs/53830
 done scraping
s

IndexError: list index out of range

In [192]:
job_listings_df["Job Title"] = job_listings_df["data"].apply(lambda x: x[0])
job_listings_df["Company Name"] = job_listings_df["data"].apply(lambda x: x[1])
job_listings_df["Seniority"] = job_listings_df["data"].apply(lambda x: x[2][0])
job_listings_df["Tech Stack"] = job_listings_df["data"].apply(lambda x: x[2][1])
job_listings_df["Job Desc"] = job_listings_df["data"].apply(lambda x: x[3])
# job_listings_df["Date posted"] = job_listings_df["Date posted"].apply(getDatePosted)

job_listings_df

,URL,Date posted,data,Job Title,Company Name,Seniority,Tech Stack,Job Desc
0,/jobs/53907,about 11 hours ago,"[ReactJS Developer (Full Stack), Apar Technolo...",ReactJS Developer (Full Stack),Apar Technologies,"[Mid, Junior]","[Docker, CloudFoundry, Spring, SonarQube, TDD,...",We are looking for a candidate to fill in the ...
1,/jobs/53898,about 11 hours ago,"[Manager, SRE, Rakuten Viki, ([Manager], [Dock...","Manager, SRE",Rakuten Viki,[Manager],"[Docker, API, PagerDuty, GKE, ELK, Sprint, UNI...",The SRE team at Viki is responsible for buildi...
2,/jobs/53894,about 11 hours ago,"[DevOps Engineer, GovTech, ([Junior], [Docker,...",DevOps Engineer,GovTech,[Junior],"[Docker, DockerCompose, Fluentd, Clair, Packer...",Our team in GovTech works on highly impactful ...
3,/jobs/53893,about 11 hours ago,"[VP, System Analyst, United Overseas Bank Limi...","VP, System Analyst",United Overseas Bank Limited (UOB),[Senior],"[ETL, Oracle, Experian, Strategy, Teradata, Ql...",The Technology and Operations function is comp...
4,/jobs/53891,about 11 hours ago,"[DevOps & Lab Manager (SG - Edge), Dell Techno...",DevOps & Lab Manager (SG - Edge),Dell Technologies,[Manager],"[Docker, Strategy, Container, Microsoft, CI, N...",Dell Technologies is seeking an entrepreneuria...
...,...,...,...,...,...,...,...,...
2959,/jobs/46008,about 2 months ago,"[2021 Graduate Site Reliability Engineer, Sing...","2021 Graduate Site Reliability Engineer, Singa...",Salesforce,[Junior],"[Docker, Container, UNIX, Kubernetes, Bootstra...","To get the best candidate experience, please c..."
2960,/jobs/46001,about 2 months ago,"[Cybersecurity SOC Specialist (Level 2), GovTe...",Cybersecurity SOC Specialist (Level 2),GovTech,[Mid],"[TCP, play, Container, Linux]",Join us as we support Singapore’s vision of bu...
2961,/jobs/45989,about 2 months ago,"[Senior DevOps Engineer (Data & Analytics), SP...",Senior DevOps Engineer (Data & Analytics),SPH Media,[Senior],"[Docker, Analytics, CI, CodeBuild, ECS, RedShi...","As a Senior DevOps Engineer, you will be respo..."
2962,/jobs/45982,about 2 months ago,"[Cybersecurity Architect, Circles.Life, ([Seni...",Cybersecurity Architect,Circles.Life,[Senior],"[Octave, Container]",The Cybersecurity Architect provides leadershi...


In [194]:
job_listings_df

,URL,Date posted,data,Job Title,Company Name,Seniority,Tech Stack,Job Desc
0,/jobs/53907,about 11 hours ago,"[ReactJS Developer (Full Stack), Apar Technolo...",ReactJS Developer (Full Stack),Apar Technologies,"[Mid, Junior]","[Docker, CloudFoundry, Spring, SonarQube, TDD,...",We are looking for a candidate to fill in the ...
1,/jobs/53898,about 11 hours ago,"[Manager, SRE, Rakuten Viki, ([Manager], [Dock...","Manager, SRE",Rakuten Viki,[Manager],"[Docker, API, PagerDuty, GKE, ELK, Sprint, UNI...",The SRE team at Viki is responsible for buildi...
2,/jobs/53894,about 11 hours ago,"[DevOps Engineer, GovTech, ([Junior], [Docker,...",DevOps Engineer,GovTech,[Junior],"[Docker, DockerCompose, Fluentd, Clair, Packer...",Our team in GovTech works on highly impactful ...
3,/jobs/53893,about 11 hours ago,"[VP, System Analyst, United Overseas Bank Limi...","VP, System Analyst",United Overseas Bank Limited (UOB),[Senior],"[ETL, Oracle, Experian, Strategy, Teradata, Ql...",The Technology and Operations function is comp...
4,/jobs/53891,about 11 hours ago,"[DevOps & Lab Manager (SG - Edge), Dell Techno...",DevOps & Lab Manager (SG - Edge),Dell Technologies,[Manager],"[Docker, Strategy, Container, Microsoft, CI, N...",Dell Technologies is seeking an entrepreneuria...
...,...,...,...,...,...,...,...,...
2959,/jobs/46008,about 2 months ago,"[2021 Graduate Site Reliability Engineer, Sing...","2021 Graduate Site Reliability Engineer, Singa...",Salesforce,[Junior],"[Docker, Container, UNIX, Kubernetes, Bootstra...","To get the best candidate experience, please c..."
2960,/jobs/46001,about 2 months ago,"[Cybersecurity SOC Specialist (Level 2), GovTe...",Cybersecurity SOC Specialist (Level 2),GovTech,[Mid],"[TCP, play, Container, Linux]",Join us as we support Singapore’s vision of bu...
2961,/jobs/45989,about 2 months ago,"[Senior DevOps Engineer (Data & Analytics), SP...",Senior DevOps Engineer (Data & Analytics),SPH Media,[Senior],"[Docker, Analytics, CI, CodeBuild, ECS, RedShi...","As a Senior DevOps Engineer, you will be respo..."
2962,/jobs/45982,about 2 months ago,"[Cybersecurity Architect, Circles.Life, ([Seni...",Cybersecurity Architect,Circles.Life,[Senior],"[Octave, Container]",The Cybersecurity Architect provides leadershi...


In [195]:
job_listings_df.to_csv("nodeflair_jobpostings.csv")